In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
# Load Data Train csv
train = pickle.load(open('data/train_bams.p','rb'))
test = pickle.load(open('data/test_bams.p','rb'))

In [3]:
train

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds,pIC50
125,0,0.0382,0.001459,138.1194,72.933239,17,18,59,36,23,...,0.174261,0.530126,0.512059,0.408730,31.318280,174.085678,385.225921,0.674239,1.450915,5.958
74,1,-0.7970,0.635209,77.2375,41.352102,6,6,37,23,14,...,0.101150,0.592622,0.550249,0.332241,18.177672,43.726934,87.066620,0.775200,1.475112,5.854
1,0,-0.0693,0.004802,113.5328,61.298032,0,0,54,30,24,...,0.124809,0.554310,0.365546,0.392117,25.097284,103.740040,238.275628,0.709610,1.311973,6.292
110,0,-2.5774,6.642991,128.7353,70.286825,17,18,60,35,25,...,0.245975,0.488948,0.408430,0.497519,24.554292,143.408571,358.730867,0.514894,1.394896,7.921
99,1,-0.4080,0.166464,117.9068,62.001653,12,12,54,33,21,...,0.325881,0.553109,0.498577,0.513381,21.955191,131.000413,332.588534,0.377649,1.565066,5.854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0,-2.0910,4.372281,119.9176,64.300032,12,12,57,33,24,...,0.119319,0.611371,0.550971,0.478145,25.754579,105.897604,242.885706,0.720412,1.640487,6.180
95,0,-0.1589,0.025249,142.5628,78.271583,12,12,70,39,31,...,0.153062,0.639028,0.396740,0.351866,39.688457,278.796848,771.668789,0.680110,1.387634,5.585
32,0,-0.4082,0.166627,106.0673,58.637653,12,12,51,30,21,...,0.344943,0.576989,0.426777,0.404989,17.907066,96.524233,263.097835,0.277946,1.408755,8.699
124,0,-0.8291,0.687407,140.0467,75.315618,17,18,63,37,26,...,0.166500,0.557440,0.485636,0.409972,31.807655,174.995325,389.991476,0.685353,1.453048,7.229


In [4]:
# Load Pickle 
label_25 = pickle.load(open("data/desc25.pkl", "rb"))
label_25

['VCH-5',
 'VCH-6',
 'MATS6e',
 'maxdsN',
 'nHCsats',
 'maxdsCH',
 'SssssC',
 'MDEN-23',
 'nssssC',
 'nHBAcc_Lipinski',
 'AATSC8i',
 'RDF30e',
 'RDF30e',
 'RDF30e',
 'maxHCsats',
 'GATS6m',
 'ATSC2i',
 'ATSC6s',
 'TDB9v',
 'SCH-5',
 'maxHBint4',
 'n3Ring',
 'SRW3',
 'maxHBint8',
 'SHCsats']

In [5]:
x_train = train.loc[:,label_25]
x_test = test.loc[:,label_25]
y_train = train.iloc[:,[-1]]
y_test = test.iloc[:,[-1]]

In [210]:
x_test.shape

(27, 25)

In [6]:
x_train.shape, x_test.shape, y_train, y_test

((107, 25),
 (27, 25),
      pIC50
 125  5.958
 74   5.854
 1    6.292
 110  7.921
 99   5.854
 ..     ...
 68   6.180
 95   5.585
 32   8.699
 124  7.229
 131  7.259
 
 [107 rows x 1 columns],
      pIC50
 80   5.252
 104  5.367
 94   5.319
 71   6.666
 51   8.569
 3    6.310
 28   7.824
 84   6.114
 126  6.958
 103  6.292
 85   5.886
 25   8.398
 48   7.921
 10   6.393
 0    5.842
 52   5.009
 35   8.000
 98   5.444
 108  6.244
 114  8.108
 13   6.224
 40   8.432
 112  6.409
 109  7.081
 26   8.495
 43   8.456
 93   5.721)

In [7]:
mmscaler = MinMaxScaler()
mmscaler.fit(x_train)
sc_x_train = mmscaler.transform(x_train)
sc_x_test = mmscaler.transform(x_test)

In [8]:
paramgrid = {
    'C' : [0.1,1,10,100,1000],
    'gamma' : ['auto','scale'],
    'epsilon' : [0.1,1,10,100,1000]
}

In [9]:
grid = GridSearchCV(
        estimator = SVR(kernel = 'rbf'),
        param_grid = paramgrid,
        cv = 5,
        scoring = 'neg_mean_squared_error',
        n_jobs = -1
)
grid.fit(sc_x_train, y_train)

C:\Users\Bams\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'epsilon': [0.1, 1, 10, 100, 1000],
                         'gamma': ['auto', 'scale']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [10]:
grid.best_params_

{'C': 100, 'epsilon': 0.1, 'gamma': 'auto'}

In [11]:
# pickle.dump(grid.best_params_, open( "r_data/params_rbf_25.p", "wb" ))

In [12]:
# model
model = SVR(kernel = 'rbf', C = grid.best_params_['C'],
           gamma = grid.best_params_['gamma'],
           epsilon = grid.best_params_['epsilon'])
model.fit(sc_x_train, y_train)

C:\Users\Bams\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [13]:
y_train_pred = model.predict(sc_x_train)
y_test_pred = model.predict(sc_x_test)

In [14]:
y_train_pred

array([7.47326301, 5.75411748, 6.39175223, 7.06835702, 5.70907409,
       8.31283796, 7.08054153, 8.75782664, 5.63656842, 6.93021541,
       7.468549  , 6.58053188, 8.16747943, 6.57878059, 8.06680342,
       5.51602893, 7.18014331, 5.89081293, 5.62146351, 8.37294945,
       7.79860377, 8.8591596 , 7.12601442, 7.29434657, 8.98586875,
       6.29137932, 5.6693459 , 6.24812037, 8.72360489, 6.02485744,
       6.67232617, 5.66895327, 5.70197912, 5.20642332, 6.49698054,
       5.80754834, 6.45358261, 6.30768316, 6.69335208, 5.6964332 ,
       5.6680457 , 5.84809012, 6.82109522, 5.84094066, 6.35419223,
       6.02127828, 5.47803514, 5.77304103, 5.62938431, 6.42275139,
       5.59905626, 5.84485585, 6.0672137 , 5.14593764, 6.85783261,
       7.18036581, 7.98693137, 6.90223571, 5.14142724, 7.13602096,
       6.63829953, 6.84557305, 5.31482367, 6.42681884, 5.7412563 ,
       5.64424882, 8.15204706, 7.27823042, 6.28843322, 8.12851121,
       6.16103699, 6.20864753, 5.79304345, 6.05496964, 7.15842

In [15]:
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

In [16]:
train_r2

0.9115547131759221

In [17]:
# pickle.dump(train_r2, open( "r_data/train_rbf_25.p", "wb" ))

In [18]:
test_r2

0.8352034758987277

In [19]:
# pickle.dump(test_r2, open( "r_data/test_rbf_25.p", "wb" ))

In [20]:
# # Load Pickle 
params5 = pickle.load(open("r_data/params_rbf_5.p", "rb"))
train_rbf5 = pickle.load(open("r_data/train_rbf_5.p", "rb"))
test_rbf5 = pickle.load(open("r_data/test_rbf_5.p", "rb"))

params10 = pickle.load(open("r_data/params_rbf_10.p", "rb"))
train_rbf10 = pickle.load(open("r_data/train_rbf_10.p", "rb"))
test_rbf10 = pickle.load(open("r_data/test_rbf_10.p", "rb"))

params15 = pickle.load(open("r_data/params_rbf_15.p", "rb"))
train_rbf15 = pickle.load(open("r_data/train_rbf_15.p", "rb"))
test_rbf15 = pickle.load(open("r_data/test_rbf_15.p", "rb"))

params20 = pickle.load(open("r_data/params_rbf_20.p", "rb"))
train_rbf20 = pickle.load(open("r_data/train_rbf_20.p", "rb"))
test_rbf20 = pickle.load(open("r_data/test_rbf_20.p", "rb"))

params25 = pickle.load(open("r_data/params_rbf_25.p", "rb"))
train_rbf25 = pickle.load(open("r_data/train_rbf_25.p", "rb"))
test_rbf25 = pickle.load(open("r_data/test_rbf_25.p", "rb"))

In [21]:
params5, train_rbf5, test_rbf5

({'C': 1, 'epsilon': 0.1, 'gamma': 'scale'},
 0.7872373072650776,
 0.7290381696602581)

In [22]:
params10, train_rbf10, test_rbf10

({'C': 10, 'epsilon': 0.1, 'gamma': 'scale'},
 0.8866944411537919,
 0.8180514275238334)

In [23]:
params15, train_rbf15, test_rbf15

({'C': 1, 'epsilon': 0.1, 'gamma': 'scale'},
 0.8582304259323502,
 0.8184262415056142)

In [24]:
params20, train_rbf20, test_rbf20

({'C': 10, 'epsilon': 0.1, 'gamma': 'scale'},
 0.9502923015889584,
 0.8160189399841534)

In [25]:
params25, train_rbf25, test_rbf25

({'C': 100, 'epsilon': 0.1, 'gamma': 'auto'},
 0.9115547131759221,
 0.8352034758987277)